# Amazon SageMaker administration and security workshop

This notebook contains hands-on exercises for the workshop **Amazon SageMaker administration and security**. 

If you don't have internet connectivity, the following statement will timeout.

In [3]:
%pip install --upgrade pip sagemaker

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


## Import packages

In [2]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker

sagemaker.__version__

'2.143.0'

## Set constants

In [3]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "from-idea-to-prod/xgboost"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()

initialized = True

print(sm_role)

arn:aws:iam::949335012047:role/sagemaker-admin-workshop-iam-StudioRoleMLOps-1LJMIG785A20T


In [4]:
bucket_name

'sagemaker-us-east-1-949335012047'

In [5]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized

Stored 'bucket_name' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)


### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value.

In [6]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        domain_id = json.loads(f.read()).get('DomainId')
        print(f"SageMaker domain id: {domain_id}")

%store domain_id

SageMaker domain id: d-dech5fdx5938
Stored 'domain_id' (str)


## Experiment with IAM execution roles

In [7]:
# Get the current user profile execution role
sagemaker.get_execution_role()

'arn:aws:iam::949335012047:role/sagemaker-admin-workshop-iam-StudioRoleMLOps-1LJMIG785A20T'

In [8]:
# Check we don't have access to some API, for example SSM DescribeParameters
!aws ssm describe-parameters


An error occurred (AccessDeniedException) when calling the DescribeParameters operation: User: arn:aws:sts::949335012047:assumed-role/sagemaker-admin-workshop-iam-StudioRoleMLOps-1LJMIG785A20T/SageMaker is not authorized to perform: ssm:DescribeParameters on resource: arn:aws:ssm:us-east-1:949335012047:* because no identity-based policy allows the ssm:DescribeParameters action


You can experiment with other AWS services and API calls. Note that there is no internet connectivity from the Studio. You can access only AWS services for what you have VPC endpoints in the Studio VPC. Calls to AWS services without VPC endpoint will timeout, for example the following call to EC2:

In [21]:
!aws ec2 describe-vpc-endpoints

^C



Check that the execution role doesn't have access  to potentially destructive API actions, such as `DeleteDomain` or `DeleteUserProfile`.
The execution role has a Deny-policy attached to it:
```json
{
        "Action": [
            "sagemaker:CreatePresignedNotebookInstanceUrl",
            "sagemaker:*NotebookInstance",
            "sagemaker:*NotebookInstanceLifecycleConfig",
            "sagemaker:CreateUserProfile",
            "sagemaker:DeleteDomain",
            "sagemaker:DeleteUserProfile"
        ],
        "Resource": [
            "*"
        ],
        "Effect": "Deny",
        "Sid": "AmazonSageMakerDeniedServices"
    }
```

In [8]:
# Try to create a new user profile
sm_client.create_user_profile(DomainId=domain_id, UserProfileName="my-new-user-profile")

ClientError: An error occurred (AccessDeniedException) when calling the CreateUserProfile operation: User: arn:aws:sts::949335012047:assumed-role/sagemaker-admin-workshop-iam-StudioRoleMLOps-1LJMIG785A20T/SageMaker is not authorized to perform: sagemaker:CreateUserProfile on resource: arn:aws:sagemaker:us-east-1:949335012047:user-profile/d-dech5fdx5938/my-new-user-profile with an explicit deny in an identity-based policy

Now check the access to allowed API for the user profile execution role, for example `sagemaker:Describe*`

In [9]:
sm_client.describe_domain(DomainId=domain_id)

{'DomainArn': 'arn:aws:sagemaker:us-east-1:949335012047:domain/d-dech5fdx5938',
 'DomainId': 'd-dech5fdx5938',
 'DomainName': 'sagemaker-admin-workshop-domain',
 'HomeEfsFileSystemId': 'fs-0e33dc594dfbf102d',
 'Status': 'InService',
 'CreationTime': datetime.datetime(2023, 4, 2, 20, 35, 42, 179000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 4, 2, 20, 38, 30, 485000, tzinfo=tzlocal()),
 'AuthMode': 'IAM',
 'DefaultUserSettings': {'ExecutionRole': 'arn:aws:iam::949335012047:role/sagemaker-admin-workshop-iam-StudioRoleDefault-F49QJ95LURXC',
  'SecurityGroups': ['sg-094cc28a340257059'],
  'SharingSettings': {'NotebookOutputOption': 'Allowed',
   'S3OutputPath': 's3://sagemaker-studio-949335012047-x2td9ne824/sharing',
   'S3KmsKeyId': 'arn:aws:kms:us-east-1:949335012047:key/acadcfc8-a091-4d23-917e-7bf0964151a5'},
  'JupyterServerAppSettings': {'DefaultResourceSpec': {'SageMakerImageArn': 'arn:aws:sagemaker:us-east-1:081325390199:image/jupyter-server-3',
    'InstanceTyp

Check that some API calls are restricted in the Data Scientist profile, but allowed in the MLOps profile.
For example, if you launch Studio for the Data Scientist profile, the call to `servicecatalog:SearchProducts` will fail with an `AccessDenied` exception. If you in the MLOps profile, the call will succeed.

In [10]:
# Check what profile you're currently in
if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        print(f"User profile: {json.loads(f.read())['UserProfileName']}")

User profile: user2-mlops


In [79]:
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model building and training"

In [80]:
# The call to SearchProducts API fails with AccessDeniedException in data science profile, but succeed in MLOps profile
sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)

{'ProductViewSummaries': [{'Id': 'prodview-hnna6dmedcsck',
   'ProductId': 'prod-53ibyqbj2cgmo',
   'Name': 'MLOps template for model building and training',
   'Owner': 'Amazon SageMaker',
   'ShortDescription': 'Use this template to automate the model building workflow. Process data, extract features, train and test models, and register them in the model registry. The template provisions an AWS CodeCommit repository for checking in and managing code versions. You can customize the seed code and the configuration files to suit your requirements.\n\nModel building pipeline: SageMaker Pipelines\nCoderepository: AWS CodeCommit\nOrchestration: AWS CodePipeline\n',
   'Type': 'CLOUD_FORMATION_TEMPLATE',
   'HasDefaultPath': False}],
 'ProductViewAggregations': {'Owner': [{'Value': '358244920887',
    'ApproximateCount': 1}],
  'ProductType': [{'Value': 'ServiceCatalog', 'ApproximateCount': 1}],
  'Vendor': []},
 'ResponseMetadata': {'RequestId': 'c3b39c03-45ed-4154-9210-1b7000b327e9',
  'H

## Experiment with internet connectivity

In [19]:
# This call will timeout because there is no public internet connectivity
!curl checkip.amazonaws.com

curl: (28) Failed to connect to checkip.amazonaws.com port 80: Connection timed out


## Use VPC endpoint policies

In [13]:
ssm = boto3.client("ssm")

In [ ]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = md["ResourceArn"].split(":")[3]

The following code cells calls the `ssm:GetParameter` API. Test the VPC endpoint policies with the following steps:
1. Run the code - it must succeed because the user profile execution role has the corresponding SSM permissions
2. Change the VPC endpoint policy for the SSM VPC endpoint. The new policy denies any SSM API call to all principles except the MLOps execution role:
```
{
    "Statement": [
        {
            "Action": [
                "ssm:*"
            ],
            "Principal": "*",
            "Resource": [
                "*"
            ],
            "Effect": "Deny",
            "Condition": {
                "ArnNotEquals": {
                    "aws:PrincipalArn": "<MLOps execution role ARN>"
                }
            }
        }
    ]
}
```
3. Run the code again - it fails with `AccessDeniedException` because of the VPC endpoint policy

In [86]:
region="us-east-1"
account_id="949335012047"
ssm.get_parameter(Name=f"sagemaker-admin-workshop-{region}-{account_id}-kms-vpce-id")

ClientError: An error occurred (AccessDeniedException) when calling the GetParameter operation: User: arn:aws:sts::949335012047:assumed-role/sagemaker-admin-workshop-iam-StudioRoleMLOps-1LJMIG785A20T/SageMaker is not authorized to perform: ssm:GetParameter on resource: arn:aws:ssm:us-east-1:949335012047:parameter/sagemaker-admin-workshop-us-east-1-949335012047-kms-vpce-id with an explicit deny in a VPC endpoint policy

## Use network configuration in SageMaker jobs

# Shutdown kernel
Each notebook contains the following code to shutdown the notebook kernel and free up the resources. If you go back and forth between notebooks, you can keep the kernel running for the duration of the workshop. Keep an eye on the instance memory allocation. All notebooks of a specific image, in this case `Data Science`, are running on the same compute instance. The default compute instance is `ml.t3.medium` with 4GB memory. You can run out of memory on the instance if you keep multiple kernels running. You can also switch to a large instance if you run out of memory for this workshop.

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>